# Hysteresis Model Code

The code for the hysteresis model. In this notebook, as an example it's being run on the full network with no dams for the duration of storm Ciara. I make use of the @jit decorator from numba, which compiles code in order to allow it to run at speeds approaching a much faster language like C. This does not however support pandas, meaning a lot of the functions are defined by looping over numpy arrays. This is the efficient approach though, as a numerical IVP solver for this problem will need to evaluate the same functions many times, and compiling these functions will greatly reduce computational cost. <br> The imports and data needed:

In [1]:
#imports:
import math
import pickle
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from numba import jit
from datetime import datetime
from scipy.optimize import root
from scipy.integrate import solve_ivp
warnings.filterwarnings('ignore')
plt.style.use('seaborn-white')



# data - note terrain data is in a shapefile so need geopandas to unpack it
terrain = gpd.read_file('data/terrain/terrain.shp')
# set coordinate system
terrain.crs = {'init': 'espg3857'}

rain = pd.read_csv('data/edited_rain.csv', header=0, parse_dates=[0], index_col=0)  #inflow data
check = pd.read_csv('data/bowston_gauge_data.csv', header=0)  #Bowston gauge data, to check results against

widths = np.load(open('data/widths.npy','rb'))  #segment widths from checking-data.ipynb

# indexing correctly
check['datetime'] = pd.to_datetime(check['datetime'], dayfirst = True)
check = check.set_index('datetime')
rain.columns = [int(x)-1 for x in rain.columns]  #correcting column index

This code solves the following system of differential equations, where $\epsilon$ is chosen to be small. For details see the writeup.
$$
\epsilon\frac{d\mathbf{h}}{dt} = \mathbf{V} - \mathbf{\tilde{V}}(\mathbf{h})
$$
$$
\frac{d\mathbf{V}}{dt} = A^\mathsf{T}\mathbf{\tilde{Q}(\mathbf{h})} - \mathbf{\tilde{Q}(\mathbf{h})} + \mathbf{q}(t)
$$
There are various imputs for the model, specifically determining the placement of dams and their properties.

In [2]:
manning = 0.03


N = terrain.shape[0]
cat_lambda = 10 * np.ones(N)

Adj = np.zeros((N,N)) #adjacency matrix
for i in range(N):
    for j in range(N):
        if terrain['TONODE'][i] == terrain['FROMNODE'][j]:
            Adj[i,j]=1

dams = []
dams = []  #Index for segments to have dams. no dams for this example run
lower_heights = []  #specifics of each dam. Must be same length as dams
upper_heights = []
kvals = []  #permeabilities - assumed to be 0 for each dam 

# timeframe of interest - this is for storm Ciara
start = pd.to_datetime('2020-02-08 6pm')
end = pd.to_datetime('2020-02-10 6pm')
eval_no = 100  #no. of points to evaluate solution at

General data handling:

In [3]:

data_df=terrain[['Shape_Leng','Slope']]
data_df.columns = ['l', 'S']
data_df['S'] = data_df['S'].apply(lambda x: x*-1)
data_df.loc[data_df.S<=0.01, 'S'] = 0.01

data_df['w']=widths
data_df['h_l']=100*np.ones(N)
data_df['h_u']=101*np.ones(N)
data_df['k']=np.zeros(N)

for i, dam in enumerate(dams):
    data_df.k[dam] = kvals[i]
    data_df.h_l[dam] = lower_heights[i]
    data_df.h_u[dam] = upper_heights[i]

Making a function to get rain values at a time $t$ seconds after the start of the interval of interest. Note that the rain function is dimensional.

In [4]:
check = check[start:end]
rain = rain[start:end]
rainvals = rain.to_numpy()

@jit(nopython=True)
def rainfun(t):
    minutes = t/60
    # find number of 5 minute intervals that have passed:
    i = int((minutes - (minutes%15))/15)
    return rainvals[i,]

Nondimensionalising - see writeup for details:

In [5]:
# constants
manning = 0.035
cst_w, cst_h, cst_S, cst_g, cst_l = 2., 1, 0.05, 9.8, 100.
cst_Q = cst_w*cst_h**(5/3)*cst_S**(1/2) / manning
cst_t = cst_l*cst_w*cst_h/cst_Q
cst_V = cst_l*cst_w*cst_h
cst_alpha = cst_h / cst_w
cst_gamma = math.sqrt(2*cst_g) * cst_h**(3/2) * cst_w / cst_Q
cst_beta = cst_h / (cst_S*cst_l)

# putting everything into a dataframe
df = pd.DataFrame()
df['l'] = data_df['l']/cst_l
df['S'] = data_df['S']/cst_S
df['w'] = data_df['w']/cst_w
df['h_l'] = data_df['h_l']/cst_h
df['h_u'] = data_df['h_u']/cst_h
df['k'] = data_df['k']
df = df.apply(pd.to_numeric, errors='coerce')

#immediately leave pandas for compatibility with numba.jit :(
l = tuple(df.l.tolist())
S = tuple(df.S.tolist())
w = tuple(df.w.tolist())
h_l = tuple(df.h_l.tolist())
h_u = tuple(df.h_u.tolist())
k = tuple(df.k.tolist())


Defining the normalised flow function $\mathbf{\tilde{Q}(\mathbf{h})}$. First, the critical values $h_c$ have to be calculated:

In [6]:
# define qfric and qdam
def Qfric1(h, i):
    Qf = w[i]*h**(5/3) * math.sqrt(S[i])
    return Qf

def Qdam1(h, i):
    a1 = h_l[i]*h / np.sqrt(h+h_l[i])
    a2 = k[i] * np.sqrt(h) * min( h-h_l[i], h_u[i]-h_l[i] )
    a3 = 2/3 * max(0, h-h_u[i])**(3/2)
    Qd = cst_gamma * w[i] * (a1+a2+a3)
    return  Qd


# find hstar values by using scipy's root solver.
# Here, finding the difference by finding roots of fun(z).

hcrit=[]
for i in range(N):
    def fun(z):
        fric = Qfric1(z, i)
        dam = Qdam1(z, i)
        return fric-dam
    hcrit.append(root(fun, h_u[i]).x[0])
hcrit = tuple(hcrit)

Defining $\mathbf{\tilde{Q}(\mathbf{h})}$ and $\mathbf{\tilde{V}(\mathbf{h})}$ properly. Note these are not vector valued functions, instead are a family of functions indexed by segment number $i$. See writeup for explanation of functions used.

In [7]:
@jit(nopython=True)
def Qfric(h, i):
    Qf = w[i]*h**(5/3) * math.sqrt(S[i])
    return Qf


@jit(nopython=True)
def Qdam(h, i):
    a1 = h_l[i]*h / np.sqrt(h+h_l[i])
    a2 = k[i] * np.sqrt(h) * min( h-h_l[i], h_u[i]-h_l[i] )
    a3 = 0.6*2/3 * max(0, h-h_u[i])**(3/2)
    Qd = cst_gamma * w[i] * (a1+a2+a3)
    return  Qd


@jit(nopython=True)
def Qfun(h, i):
    if h<max(hcrit[i], h_l[i]):
        return Qfric(h, i)
    else:
        return Qdam(h, i)

@jit(nopython=True)        
def Vfun(h, i):
    hstr = (Qfun(h,i) /w[i] /np.sqrt(S[i]))**(3/5)  #downstream flow
    if h < hstr + S[i]*l[i]/cst_beta:
        return w[i]*l[i]*hstr + cst_lambda[i]*0.5*cst_beta*w[i]* max(0, h-hstr)**2 / S[i]
    else:
        return w[i]*l[i]*( h - S[i]*l[i]/cst_beta + cst_lambda[i]/2*S[i]*l[i]/cst_beta)
  

Define the derivative $D$ at a point. Note this is a derivative for both $\mathbf{h}$ and $\mathbf{V}$. Note, here $\epsilon = 0.1$.

In [8]:
@jit(nopython=True)
def fun(h, V, q):
    Q = np.zeros((N,1))
    for i in range(N):
        Q[i,0] = Qfun(h[i],i)
    # dh/dt
    D1 = [ 10*(V[i] - Vfun(h[i],i)) for i in range(N) ] 
    # dV/dt
    D2 = Adj.T.dot(Q) - Q  + q
    D2 = list(D2.flat)
    return D1 + D2

@jit(nopython=True)
def derivative(t, X):
    #unpacking
    h = X[:N]
    V = X[N:]
    q = np.zeros((N,1))
    q[:,0] = rainfun(t*cst_t)/cst_Q
    return fun(h, V, q)

To solve the problem, I am using scipy's standard IVP solver. As this can be a stiff problem ($\mathbf{h}$ varies on a smaller timescale than $\mathbf{V}$), I'm using an implicit solver designed for stiff problems. To get suitable initial conditions, I'm running it at a baseline for a short time.

In [9]:
T = (end-start).total_seconds() / cst_t

def derivative2(t,X):
    return derivative(0,X)

initial = [0]*(2*N)
sol = solve_ivp(derivative2, (0,60*60/cst_t), initial, atol=1e-2, rtol=1e-2)
initial = sol.y[:,-1]


T = (end-start).total_seconds() / cst_t
startTime = datetime.now()
print('Final time is '+str(T))
sol = solve_ivp(derivative, (0,T), initial, t_eval=np.linspace(0,T,eval_no) , atol=1e-3, rtol=1e-3, method='RK23')
print('The total time taken is ' + str((datetime.now()-startTime).total_seconds())+' seconds.')

Final time is 11039.787043198961
The total time taken is 10779.229582 seconds.


The most useful output are flow values. These can be calculated from Qfun defined above. I'm saving the flow values to a csv.

In [11]:
# apply Q(h) to the array of h values
qoutput = pd.DataFrame(sol.y[:N].T, index=pd.date_range(start, end, 100))  #h solution values    

for i, col in enumerate(qoutput.columns):
    qoutput[col] = qoutput[col].apply(lambda x: Qfun(x, i)*cst_Q)  #note renormalisation
qoutput.to_csv('data/hysterisis_flow_data.csv')